In [27]:
pip install PyDrive

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install requests google-auth-oauthlib google-auth-httplib2

Note: you may need to restart the kernel to use updated packages.


In [33]:
import requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# OAuth 2.0 클라이언트 ID를 사용하여 인증
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# OAuth 2.0 인증 설정
def authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

# Google Drive API 호출
def list_files(creds):
    url = 'https://www.googleapis.com/drive/v3/files'
    headers = {'Authorization': f'Bearer {creds.token}'}
    response = requests.get(url, headers=headers)
    return response.json()

creds = authenticate()
files = list_files(creds)
print(files)


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [32]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
# Google OAuth 인증 절차
gauth = GoogleAuth()
gauth.CommandLineAuth()  # 이 과정을 통해 브라우저가 열리고, Google 계정 인증이 이루어짐

drive = GoogleDrive(gauth)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=20853731651-6grrfq6n7htqhferl1ah8imldetrgg53.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



Enter verification code:  4/1AQlEd8xhQfo6A4pYib9WK4rEN98ZXdMQMgbNy8my_L2t5nQCHOQ6XXmI4Q4


Authentication successful.


In [39]:
# import os
# folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# for file in file_list[:10]:
#     print(f"Downloading {file['title']} from {file['id']}")

In [44]:
# Google Drive에서 이미지와 라벨 파일을 다운로드하여 로컬 경로로 저장
def download_drive_folder(folder_id, local_path):
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    if not os.path.exists(local_path):
        os.makedirs(local_path)
    
    for file in file_list:
        # MIME 타입 확인 (예: 이미지 파일만 다운로드)
        if 'application/vnd.google-apps' not in file['mimeType']:  # Google Docs 등은 제외
            print(f"Downloading {file['title']} from Google Drive")
            file.GetContentFile(os.path.join(local_path, file['title']))
        else:
            print(f"Skipping non-downloadable file: {file['title']}")

# Google Drive에서 폴더 ID를 이용해 데이터 다운로드
image_folder_id = '1Nby_3Fg7NZ-CpLDjimVm9cC3n3imW4Sd'  # Google Drive의 이미지 폴더 ID 입력
label_folder_id = '17KDAGcw4JLD3COQHQVYk68AyxUysTGPA'  # Google Drive의 라벨 폴더 ID 입력

image_local_path = './images'
label_local_path = './labels'

# 이미지 및 라벨 다운로드
download_drive_folder(image_folder_id, image_local_path)
download_drive_folder(label_folder_id, label_local_path)

# LaneDataset 클래스 정의
class LaneDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_folders = os.listdir(image_dir)
        self.label_folders = os.listdir(label_dir)

        # 각 폴더 안에 있는 이미지 파일들을 미리 저장
        self.image_files = []
        for folder in self.image_folders:
            folder_path = os.path.join(self.image_dir, folder)
            images = os.listdir(folder_path)  # 이미지 파일 리스트
            for img_file in images:
                self.image_files.append((folder, img_file)) # (폴더명, 이미지 파일명) 저장

    def __len__(self):
        return len(self.image_files)  # 모든 이미지 파일의 총 개수

    def __getitem__(self, idx):
        # 이미지 파일 정보
        folder, img_file = self.image_files[idx]
        img_path = os.path.join(self.image_dir, folder, img_file)

        # 해당 이미지 파일에 대응하는 라벨 폴더 및 라벨 파일
        label_folder_path = os.path.join(self.label_dir, folder)
        label_file = os.listdir(label_folder_path)[0]  # 라벨 파일 선택
        label_path = os.path.join(label_folder_path, label_file)

        # 이미지 로드
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 라벨 로드 (JSON 파일에서 차선 종류 및 위반 여부 정보 추출)
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # 위반 여부 및 차선 종류 추출
        violation, lane_type = self.extract_violation(label_path)

        # 필요한 경우 데이터 변환 적용
        if self.transform:
            image = self.transform(image)

        return image, violation, lane_type

    # JSON 파일에서 위반 여부와 차선 종류 추출 함수
    def extract_violation(self, json_file_path):
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # 위반 여부와 차선 종류 기본값 설정
        violation = False  # 기본적으로는 위반 아님
        lane_type = None   # 차선 종류는 None을 기본값으로 설정

        # "data_set_info"에서 "data" 항목에 있는 객체 정보 확인
        for obj in data['data_set_info']['data']:
            object_label = obj['value']['object_Label']

            # 위반 여부 판단
            if 'vehicle_attribute' in object_label and object_label['vehicle_attribute'] == 'violation':
                violation = True  # 위반 차량 있음
            # 차선 종류 판단
            if 'lane_type' in object_label:
                lane_type = object_label['lane_type']

        return violation, lane_type

# 데이터 변환 설정
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # ResNet 입력 크기와 맞추기
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 이미지와 라벨 데이터셋 생성
dataset = LaneDataset(image_dir=image_local_path, label_dir=label_local_path, transform=data_transform)
data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# DataLoader에서 데이터 순차적으로 불러오기
for images, (violations, lane_types) in data_loader:
    print(images.shape, violations, lane_types)

Skipping non-downloadable file: [25627]A_BLUE
Skipping non-downloadable file: [25625]A_BLUE
Skipping non-downloadable file: [25624]A_BLUE
Skipping non-downloadable file: [25623]A_BLUE
Skipping non-downloadable file: [25621]A_BLUE
Skipping non-downloadable file: [25619]A_BLUE
Skipping non-downloadable file: [25618]A_BLUE
Skipping non-downloadable file: [25617]A_BLUE
Skipping non-downloadable file: [25593]A_BLUE
Skipping non-downloadable file: [25591]A_BLUE
Skipping non-downloadable file: [25590]A_BLUE
Skipping non-downloadable file: [25589]A_BLUE
Skipping non-downloadable file: [25588]A_BLUE
Skipping non-downloadable file: [25587]A_BLUE
Skipping non-downloadable file: [25584]A_BLUE
Skipping non-downloadable file: [25583]A_BLUE
Skipping non-downloadable file: [25580]A_BLUE
Skipping non-downloadable file: [25579]A_BLUE
Skipping non-downloadable file: [25577]A_BLUE
Skipping non-downloadable file: [25574]A_BLUE
Skipping non-downloadable file: [25572]A_BLUE
Skipping non-downloadable file: [2

NameError: name 'Dataset' is not defined